In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Activation, Conv2DTranspose, MaxPool2D, Input
from tensorflow.keras.models import Sequential
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from IPython.display import Image, display
from skimage.transform import resize
from util import color_to_label, label_to_colors

In [2]:
# 1903 w
# 2981 himport os
# IMAGE_SIZE = (1903, 2981)
IMAGE_SIZE = (256, 256)

input_dir = "./GW5060/binary"
target_dir = "./GW5060/masks/"
img_size = (160, 160)
num_classes = 3
batch_size = 32

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".tif")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)


Number of samples: 10
./GW5060/binary/BE_1405_0010.tif | ./GW5060/masks/BE_1405_0010_MASK.png
./GW5060/binary/BE_1405_0011.tif | ./GW5060/masks/BE_1405_0011_MASK.png
./GW5060/binary/BE_1405_0012.tif | ./GW5060/masks/BE_1405_0012_MASK.png
./GW5060/binary/BE_1405_0013.tif | ./GW5060/masks/BE_1405_0013_MASK.png
./GW5060/binary/BE_1405_0014.tif | ./GW5060/masks/BE_1405_0014_MASK.png
./GW5060/binary/BE_1405_0015.tif | ./GW5060/masks/BE_1405_0015_MASK.png
./GW5060/binary/BE_1405_0016.tif | ./GW5060/masks/BE_1405_0016_MASK.png
./GW5060/binary/BE_1405_0017.tif | ./GW5060/masks/BE_1405_0017_MASK.png
./GW5060/binary/BE_1405_0018.tif | ./GW5060/masks/BE_1405_0018_MASK.png
./GW5060/binary/BE_1405_0019.tif | ./GW5060/masks/BE_1405_0019_MASK.png


In [3]:
image2 = PIL.Image.open(input_img_paths[0])
image2_mask = PIL.Image.open(target_img_paths[0])

In [4]:
img_numpy = np.array(image2)
img_numpy = resize(img_numpy, IMAGE_SIZE)
img_numpy = img_numpy.reshape(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)
img_numpy.shape


(256, 256, 1)

In [5]:
X_train = np.zeros((len(input_img_paths), IMAGE_SIZE[0], IMAGE_SIZE[1],1))
y_train = np.zeros((len(target_img_paths), IMAGE_SIZE[0], IMAGE_SIZE[1]))
X_train.shape
y_train.shape

(10, 256, 256)

In [6]:
for idx, image in enumerate(input_img_paths):
    img = PIL.Image.open(image)
    img = np.array(img)
    img = resize(img, IMAGE_SIZE)
    img = img.reshape(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)
    # print(img.shape)
    X_train[idx] = img

for idx, image in enumerate(target_img_paths):
    label_img = PIL.Image.open(image)
    label_img = np.array(label_img)
    label_img = resize(label_img, IMAGE_SIZE)
    # print(label_img.shape)
    # label_img = label_img.reshape(IMAGE_SIZE[0], IMAGE_SIZE[1],3)
    # print(label_img.shape)
    label_img = color_to_label(label_img)
    y_train[idx] = label_img

In [7]:
ks = (3,3)
model = Sequential([
    # Encoder
    Input(shape=(256,256,1)),
    Conv2D(20, ks, padding='same'),
    Activation('relu'),
    Conv2D(40, ks, padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(60, ks, padding='same'),
    Activation('relu'),
    Conv2D(80, ks, padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(100, ks, padding='same'),
    Activation('relu'),
    # Decoder
    Conv2DTranspose(80, ks, padding='same'),
    Activation('relu'),
    Conv2DTranspose(60, ks, strides=(2,2),padding='same'),
    Activation('relu'),   
    Conv2DTranspose(40, ks, strides=(2,2),padding='same'),
    Conv2DTranspose(5, kernel_size=(1,1)),
    Activation('softmax'),
    ])

In [8]:
model.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 20)      200       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 20)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 40)      7240      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 40)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 60)      21660     
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 60)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 80)      4

In [9]:

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=2)

Epoch 1/2
1/1 [==============================] - 11s 11s/step - loss: 1.6049 - accuracy: 0.2463
Epoch 2/2
1/1 [==============================] - 0s 200ms/step - loss: 1.5515 - accuracy: 0.8733


In [11]:
y_pred = model.predict(X_train)

In [12]:
model.evaluate(X_train, y_train)

1/1 [==============================] - 0s 251ms/step - loss: 1.3906 - accuracy: 0.9989


[1.3906495571136475, 0.998919665813446]

In [20]:
y_pred.shape

(10, 256, 256, 5)

In [14]:
y_pred_2 = label_to_colors(y_pred)

In [16]:
y_pred_2.shape

(10, 256, 256, 5, 3)

In [19]:
img_example = PIL.Image(y_pred_2[0])

TypeError: 'module' object is not callable